## ResNet50 Backbone with Frozen Weights

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.transforms.functional as VF
from torchvision import transforms

import os, glob
import numpy as np
from PIL import Image

# ------------------------------
# Arguments
# ------------------------------
class Args:
    num_classes = 3
    batch_size = 64  # Can increase since we're just doing inference
    num_workers = 4
    backbone = "resnet50"  # Changed to ResNet50
    
    # Paths - CHANGE THESE
    dataset = "/projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/val_tiles/*"
    output  = "./feature_extractor/graphs_all"  # This matches your GTP expected path
    
args = Args()

# ------------------------------
# Helper classes/functions
# ------------------------------
class BagDataset():
    def __init__(self, csv_file, transform=None):
        self.files_list = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.files_list)

    def __getitem__(self, idx):
        temp_path = self.files_list[idx]
        img = Image.open(temp_path).resize((224, 224))
        sample = {'input': img}
        if self.transform:
            sample = self.transform(sample)
        return sample

class ToTensor(object):
    def __call__(self, sample):
        img = sample['input']
        img = VF.to_tensor(img)
        # Add ImageNet normalization
        img = VF.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return {'input': img}

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

def save_coords(txt_file, csv_file_path):
    for path in csv_file_path:
        parts = path.split('/')[-1].split('.')[0].split('_')
        if len(parts) >= 2:
            x, y = parts[0], parts[1]
            txt_file.write(f"{x}\t{y}\n")
    txt_file.close()

def adj_matrix(csv_file_path, _):
    total = len(csv_file_path)
    adj_s = np.zeros((total, total))

    for i in range(total-1):
        parts_i = csv_file_path[i].split('/')[-1].split('.')[0].split('_')
        if len(parts_i) < 2:
            continue
        xi, yi = parts_i[0], parts_i[1]
        
        for j in range(i+1, total):
            parts_j = csv_file_path[j].split('/')[-1].split('.')[0].split('_')
            if len(parts_j) < 2:
                continue
            xj, yj = parts_j[0], parts_j[1]
            
            if abs(int(xi)-int(xj)) <= 1 and abs(int(yi)-int(yj)) <= 1:
                adj_s[i][j] = 1
                adj_s[j][i] = 1

    return torch.tensor(adj_s).cuda()

def bag_dataset(args, csv_file_path):
    ds = BagDataset(csv_file=csv_file_path, transform=Compose([ToTensor()]))
    loader = DataLoader(ds, batch_size=args.batch_size, shuffle=False,
                        num_workers=args.num_workers, drop_last=False)
    return loader, len(ds)

def compute_feats(args, bags_list, feature_extractor, save_path=None):
    """Extract features and build graphs."""
    
    for i, bag_path in enumerate(bags_list):
        # Get all patch files
        csv_file_path = []
        for ext in ("*.png", "*.jpeg", "*.jpg"):
            csv_file_path.extend(glob.glob(os.path.join(bag_path, ext)))
        
        file_name = bag_path.split("/")[-1]
        print(f"\n[{i+1}/{len(bags_list)}] Processing: {file_name}")
        print(f"  Found {len(csv_file_path)} patches")

        # Output directory (changed to match GTP expected structure)
        out_dir = os.path.join(save_path, "panda", file_name)
        
        # Skip if already processed or no patches
        if os.path.isdir(out_dir):
            print("  Already exists — skipping.")
            continue
        
        if len(csv_file_path) < 1:
            print("  No patches — skipping.")
            continue

        # Create dataloader
        dataloader, _ = bag_dataset(args, csv_file_path)

        # Extract features
        feats_list = []
        with torch.no_grad():
            for batch_idx, batch in enumerate(dataloader):
                patches = batch["input"].float().cuda()
                feats = feature_extractor(patches)
                
                # Flatten if needed
                if feats.dim() > 2:
                    feats = feats.squeeze(-1).squeeze(-1)
                
                feats_list.append(feats)
                
                if (batch_idx + 1) % 10 == 0:
                    processed = min((batch_idx + 1) * args.batch_size, len(csv_file_path))
                    print(f"    Processed {processed}/{len(csv_file_path)} patches")

        # Stack all features
        features = torch.cat(feats_list, dim=0).cuda()
        print(f"  Total features: {features.shape}")

        # Save outputs
        os.makedirs(out_dir, exist_ok=True)

        # Save coordinates
        txt_file = open(os.path.join(out_dir, "c_idx.txt"), "w+")
        save_coords(txt_file, csv_file_path)

        # Save features
        torch.save(features, os.path.join(out_dir, "features.pt"))

        # Build and save adjacency matrix
        adj_s = adj_matrix(csv_file_path, features)
        torch.save(adj_s, os.path.join(out_dir, "adj_s.pt"))
        
        print(f"  ✓ Saved to {out_dir}")

# ------------------------------
# Load ResNet50 Backbone (ImageNet Pretrained)
# ------------------------------
print("="*70)
print("LOADING RESNET50 BACKBONE (IMAGENET PRETRAINED)")
print("="*70)

# Load ImageNet pretrained ResNet50
feature_extractor = models.resnet50(weights='IMAGENET1K_V1')
# Remove classification head
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])

# Move to GPU and freeze
feature_extractor = feature_extractor.cuda()
feature_extractor.eval()

for param in feature_extractor.parameters():
    param.requires_grad = False

print("✓ ResNet50 loaded")
print("  Feature dimension: 2048")
print("  Pretrained: ImageNet")
print("="*70)
print()

# ------------------------------
# Build Graphs
# ------------------------------
print("Starting graph construction...")
print(f"Input: {args.dataset}")
print(f"Output: {args.output}")
print()

os.makedirs(args.output, exist_ok=True)

bags_list = glob.glob(args.dataset)
print(f"Found {len(bags_list)} WSIs to process")
print()

if len(bags_list) == 0:
    print("⚠ No WSIs found! Check your dataset path.")
else:
    compute_feats(args, bags_list, feature_extractor, args.output)
    
    print()
    print("="*70)
    print("✓ GRAPH BUILDING COMPLETE!")
    print("="*70)
    print(f"Graphs saved to: {args.output}/panda/")
    print()
    print("Next steps:")
    print("1. Process remaining subsets (tiles_02 through tiles_10)")
    print("2. Train graph transformer with these graphs")
    print("3. Evaluate on validation set")

LOADING RESNET50 BACKBONE (IMAGENET PRETRAINED)
✓ ResNet50 loaded
  Feature dimension: 2048
  Pretrained: ImageNet

Starting graph construction...
Input: /projectnb/ec500kb/projects/Project_1_Team_1/PANDA_DATA_MANNY/val_tiles/*
Output: ./feature_extractor/graphs_all

Found 2124 WSIs to process


[1/2124] Processing: 848c7be5504d4f2e9ba4e949c619ef62
  Found 182 patches
  Total features: torch.Size([182, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/848c7be5504d4f2e9ba4e949c619ef62

[2/2124] Processing: ada83813085876c62c8bf0481f1693cf
  Found 78 patches
  Total features: torch.Size([78, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/ada83813085876c62c8bf0481f1693cf

[3/2124] Processing: cc6cc8edfde85b399233cf8d891de0a0
  Found 138 patches
  Total features: torch.Size([138, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/cc6cc8edfde85b399233cf8d891de0a0

[4/2124] Processing: ae1c6bd846449adbde74b0cba5cb7a19
  Found 177 patches
  Total features: torch.Size([1

  Total features: torch.Size([73, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/ee83e1603d89100f6546368fe275aaa5

[40/2124] Processing: a6ba6423d654ae961d730c72a1a2f50f
  Found 138 patches
  Total features: torch.Size([138, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/a6ba6423d654ae961d730c72a1a2f50f

[41/2124] Processing: 375b2c9501320b35ceb638a3274812aa
  Found 121 patches
  Total features: torch.Size([121, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/375b2c9501320b35ceb638a3274812aa

[42/2124] Processing: 9e994536fd365604921e85f145f6e7b5
  Found 135 patches
  Total features: torch.Size([135, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/9e994536fd365604921e85f145f6e7b5

[43/2124] Processing: 3ef789f8caa7186261ad9f77bb552c7a
  Found 141 patches
  Total features: torch.Size([141, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/3ef789f8caa7186261ad9f77bb552c7a

[44/2124] Processing: de16d4d453830a7b68b20e591f41bc6c
  Found 159 patc

  Total features: torch.Size([159, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/e02703af0349a0d093605123400dbf07

[81/2124] Processing: 83ebc436de663c1259561ecec01d96ed
  Found 422 patches
  Total features: torch.Size([422, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/83ebc436de663c1259561ecec01d96ed

[82/2124] Processing: 39f81a4d7b5b86706391dcc09a4c2e94
  Found 99 patches
  Total features: torch.Size([99, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/39f81a4d7b5b86706391dcc09a4c2e94

[83/2124] Processing: 2f0131a5e37bdf341c47dfc724eb0eb2
  Found 170 patches
  Total features: torch.Size([170, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/2f0131a5e37bdf341c47dfc724eb0eb2

[84/2124] Processing: 0841bb278823ec53920d723881c7afd9
  Found 145 patches
  Total features: torch.Size([145, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/0841bb278823ec53920d723881c7afd9

[85/2124] Processing: f1079a2a43462e7cbaa60faec097b2f6
  Found 118 patch

  Total features: torch.Size([112, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/4ca21d08bb5a350a586fca9867943535

[123/2124] Processing: d4f83e0d9b46e89cd5e9da401f625bce
  Found 191 patches
  Total features: torch.Size([191, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/d4f83e0d9b46e89cd5e9da401f625bce

[124/2124] Processing: 2910b44434274b848553a4ec3db11df8
  Found 189 patches
  Total features: torch.Size([189, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/2910b44434274b848553a4ec3db11df8

[125/2124] Processing: 6a7acfa9447d5df18f50e7ffdfe34f38
  Found 198 patches
  Total features: torch.Size([198, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/6a7acfa9447d5df18f50e7ffdfe34f38

[126/2124] Processing: a61bde5fa031a829eed61c44ea348963
  Found 94 patches
  Total features: torch.Size([94, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/a61bde5fa031a829eed61c44ea348963

[127/2124] Processing: 602dcec72143bd0f1eabfad244f081dd
  Found 79 p

  Total features: torch.Size([123, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/39e0f864a25b0498bf35d6d27e36d4f3

[164/2124] Processing: 99a41a5e8adfc6b7589ea4523484e217
  Found 146 patches
  Total features: torch.Size([146, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/99a41a5e8adfc6b7589ea4523484e217

[165/2124] Processing: b440e34f6b1bc9abd0844a7ed19a6b45
  Found 82 patches
  Total features: torch.Size([82, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/b440e34f6b1bc9abd0844a7ed19a6b45

[166/2124] Processing: 6e52a788e36a88c5267d49e219051d05
  Found 238 patches
  Total features: torch.Size([238, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/6e52a788e36a88c5267d49e219051d05

[167/2124] Processing: 42482ed25f9001ae44f6148432da91d7
  Found 168 patches
  Total features: torch.Size([168, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/42482ed25f9001ae44f6148432da91d7

[168/2124] Processing: 330823f8120df480aefdbf1d107747ba
  Found 150 

...

  Total features: torch.Size([139, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/0076bcb66e46fb485f5ba432b9a1fe8a

[1927/2124] Processing: f4252060c37bb037f3b93a39e5a4819e
  Found 87 patches
  Total features: torch.Size([87, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/f4252060c37bb037f3b93a39e5a4819e

[1928/2124] Processing: 773d5f35cb242573885d18d302a9157c
  Found 105 patches
  Total features: torch.Size([105, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/773d5f35cb242573885d18d302a9157c

[1929/2124] Processing: aea0dc89fbdd0bbca975eb440b55d3c1
  Found 114 patches
  Total features: torch.Size([114, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/aea0dc89fbdd0bbca975eb440b55d3c1

[1930/2124] Processing: 80b59404657a6290a06e8902430be7fc
  Found 169 patches
  Total features: torch.Size([169, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/80b59404657a6290a06e8902430be7fc

[1931/2124] Processing: d76be95baca47c31d827d59abbe132b6
  Found

  Total features: torch.Size([176, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/bd9f6b90049d6a63c66147e140bbc9eb

[1968/2124] Processing: ce8f7d335ba01839ce10eaff44daff42
  Found 81 patches
  Total features: torch.Size([81, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/ce8f7d335ba01839ce10eaff44daff42

[1969/2124] Processing: 0e2203e117e27e9728407f102404ac59
  Found 45 patches
  Total features: torch.Size([45, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/0e2203e117e27e9728407f102404ac59

[1970/2124] Processing: e5739e452d5b7c5e68c4171ece2557e3
  Found 216 patches
  Total features: torch.Size([216, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/e5739e452d5b7c5e68c4171ece2557e3

[1971/2124] Processing: 8addcf68a92fde8dba0b2a99caf9bff9
  Found 158 patches
  Total features: torch.Size([158, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/8addcf68a92fde8dba0b2a99caf9bff9

[1972/2124] Processing: 6437c1f6a475089d659e71ddd7edb010
  Found 1

  Total features: torch.Size([175, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/78ef82c809a7a83e145b2ff221af942d

[2009/2124] Processing: 8f18d0ffda3fb439d6ad4473d575409d
  Found 101 patches
  Total features: torch.Size([101, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/8f18d0ffda3fb439d6ad4473d575409d

[2010/2124] Processing: 2f66cb1103d6a05264fcc8e8841a6890
  Found 112 patches
  Total features: torch.Size([112, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/2f66cb1103d6a05264fcc8e8841a6890

[2011/2124] Processing: a331bd194ce409c37923a64ae652009f
  Found 169 patches
  Total features: torch.Size([169, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/a331bd194ce409c37923a64ae652009f

[2012/2124] Processing: 1075cff47acd7b66b405f2d186192ad9
  Found 138 patches
  Total features: torch.Size([138, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/1075cff47acd7b66b405f2d186192ad9

[2013/2124] Processing: afba29a2f7c57d2f2f1f0806f14001ba
  Fou

  Total features: torch.Size([103, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/38da5a625694c55fccb0c6daeb2fdd58

[2050/2124] Processing: a3da6a9b33b3bbbbc3a2121a06a75ee3
  Found 230 patches
  Total features: torch.Size([230, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/a3da6a9b33b3bbbbc3a2121a06a75ee3

[2051/2124] Processing: ef370f7b011f0c0303a7ae14fe0c0770
  Found 60 patches
  Total features: torch.Size([60, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/ef370f7b011f0c0303a7ae14fe0c0770

[2052/2124] Processing: 9d22e04259608d9e5bafa6ee02bcd803
  Found 242 patches
  Total features: torch.Size([242, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/9d22e04259608d9e5bafa6ee02bcd803

[2053/2124] Processing: 385db50add69be4288db35c092fac80f
  Found 209 patches
  Total features: torch.Size([209, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/385db50add69be4288db35c092fac80f

[2054/2124] Processing: bec80e876b9ffa8f24cf85beaa431667
  Found

  Total features: torch.Size([77, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/27e5bb4709a5b4e8dd9f7c7a3f45fa2d

[2091/2124] Processing: 0d1e1a99a3ba55fe2d5cf69ba87c9d97
  Found 131 patches
  Total features: torch.Size([131, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/0d1e1a99a3ba55fe2d5cf69ba87c9d97

[2092/2124] Processing: 7ae412eda1a4203bb31484fdcdf0864d
  Found 169 patches
  Total features: torch.Size([169, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/7ae412eda1a4203bb31484fdcdf0864d

[2093/2124] Processing: 659d4fca38eed915a13188a1f2b03ee4
  Found 149 patches
  Total features: torch.Size([149, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/659d4fca38eed915a13188a1f2b03ee4

[2094/2124] Processing: 5bd19956ab63289395ba2b259a9aa8d1
  Found 103 patches
  Total features: torch.Size([103, 2048])
  ✓ Saved to ./feature_extractor/graphs_all/panda/5bd19956ab63289395ba2b259a9aa8d1

[2095/2124] Processing: 4cc1691f988cd4596b2a38d1e64c18ce
  Foun

Testing Node Features from  ./feature_extractor/graphs_all*

In [34]:
#!/usr/bin/env python3
"""
Quick inspection of graph node features from subset 1
"""
import torch
import numpy as np
import os
from glob import glob

print("="*70)
print("GRAPH FEATURE INSPECTION - SUBSET 1")
print("="*70)

# Path to graphs
graph_dir = './feature_extractor/graphs_all/panda'

# Get all graphs
all_graphs = sorted(os.listdir(graph_dir))
print(f"\nTotal graphs available: {len(all_graphs)}")

# Sample some graphs to inspect
sample_size = min(20, len(all_graphs))
sample_graphs = np.random.choice(all_graphs, sample_size, replace=False)

print(f"Inspecting {sample_size} random graphs...\n")

# Collect statistics
all_node_features = []
graph_stats = []

for graph_id in sample_graphs:
    graph_path = os.path.join(graph_dir, graph_id)
    
    # Load features
    features_file = os.path.join(graph_path, 'features.pt')
    adj_file = os.path.join(graph_path, 'adj_s.pt')
    
    if not os.path.exists(features_file):
        continue
    
    features = torch.load(features_file, map_location='cpu')
    adj = torch.load(adj_file, map_location='cpu')
    
    # Calculate stats for this graph
    num_nodes = features.shape[0]
    feature_dim = features.shape[1]
    num_edges = adj.sum().item() / 2  # Divide by 2 because symmetric
    
    node_means = features.mean(dim=1)  # Mean across features for each node
    node_stds = features.std(dim=1)    # Std across features for each node
    
    graph_stats.append({
        'id': graph_id,
        'num_nodes': num_nodes,
        'feature_dim': feature_dim,
        'num_edges': num_edges,
        'node_feature_mean': features.mean().item(),
        'node_feature_std': features.std().item(),
        'node_feature_min': features.min().item(),
        'node_feature_max': features.max().item(),
    })
    
    # Collect all features for global statistics
    all_node_features.append(features)

# Stack all features
all_features = torch.cat(all_node_features, dim=0)  # [total_nodes, 2048]

print("="*70)
print("PER-GRAPH STATISTICS")
print("="*70)
print(f"{'Graph ID':<45} {'Nodes':>6} {'Edges':>8} {'Feat Mean':>10} {'Feat Std':>10}")
print("-"*70)

for stats in graph_stats[:10]:  # Show first 10
    print(f"{stats['id'][:45]:<45} "
          f"{stats['num_nodes']:>6} "
          f"{int(stats['num_edges']):>8} "
          f"{stats['node_feature_mean']:>10.4f} "
          f"{stats['node_feature_std']:>10.4f}")

if len(graph_stats) > 10:
    print(f"... and {len(graph_stats) - 10} more graphs")

print("\n" + "="*70)
print("GLOBAL STATISTICS (All Sampled Graphs)")
print("="*70)

# Node counts
node_counts = [s['num_nodes'] for s in graph_stats]
print(f"\nNodes per graph:")
print(f"  Mean:   {np.mean(node_counts):.1f}")
print(f"  Median: {np.median(node_counts):.1f}")
print(f"  Min:    {np.min(node_counts)}")
print(f"  Max:    {np.max(node_counts)}")

# Edge counts
edge_counts = [s['num_edges'] for s in graph_stats]
print(f"\nEdges per graph:")
print(f"  Mean:   {np.mean(edge_counts):.1f}")
print(f"  Median: {np.median(edge_counts):.1f}")
print(f"  Min:    {np.min(edge_counts):.1f}")
print(f"  Max:    {np.max(edge_counts):.1f}")

# Feature statistics
print(f"\n" + "="*70)
print("NODE FEATURE STATISTICS (2048-dim ResNet50)")
print("="*70)

print(f"\nTotal nodes sampled: {all_features.shape[0]}")
print(f"Feature dimension:   {all_features.shape[1]}")

print(f"\nNode feature values:")
print(f"  Mean:  {all_features.mean():.6f}")
print(f"  Std:   {all_features.std():.6f}")
print(f"  Min:   {all_features.min():.6f}")
print(f"  Max:   {all_features.max():.6f}")

# Per-feature-dimension statistics
feature_means = all_features.mean(dim=0)  # Mean of each of 2048 features
feature_stds = all_features.std(dim=0)    # Std of each of 2048 features

print(f"\nPer-dimension statistics (across all nodes):")
print(f"  Feature dim means - Mean: {feature_means.mean():.6f}, Std: {feature_means.std():.6f}")
print(f"  Feature dim stds  - Mean: {feature_stds.mean():.6f}, Std: {feature_stds.std():.6f}")

# Check for dead features (dimensions that are always zero or constant)
dead_features = (feature_stds < 1e-6).sum().item()
print(f"\nDead features (std < 1e-6): {dead_features} / {all_features.shape[1]}")

# Feature norm distribution
feature_norms = torch.norm(all_features, dim=1)  # L2 norm per node
print(f"\nNode feature norms (L2):")
print(f"  Mean:   {feature_norms.mean():.4f}")
print(f"  Median: {feature_norms.median():.4f}")
print(f"  Min:    {feature_norms.min():.4f}")
print(f"  Max:    {feature_norms.max():.4f}")

# Cosine similarity between random pairs (diversity check)
print(f"\n" + "="*70)
print("FEATURE DIVERSITY CHECK")
print("="*70)

from scipy.spatial.distance import cosine
num_samples = min(100, all_features.shape[0])
sample_indices = np.random.choice(all_features.shape[0], num_samples, replace=False)
sampled_features = all_features[sample_indices].numpy()

similarities = []
for i in range(len(sampled_features)):
    for j in range(i+1, min(i+10, len(sampled_features))):  # Check 10 neighbors
        sim = 1 - cosine(sampled_features[i], sampled_features[j])
        similarities.append(sim)

similarities = np.array(similarities)
print(f"\nPairwise cosine similarity (sampled):")
print(f"  Mean:   {similarities.mean():.4f}")
print(f"  Median: {similarities.median():.4f}")
print(f"  Std:    {similarities.std():.4f}")
print(f"  Min:    {similarities.min():.4f}")
print(f"  Max:    {similarities.max():.4f}")

# Interpretation
print(f"\n" + "="*70)
print("INTERPRETATION")
print("="*70)

if similarities.mean() < 0.3:
    print("✓ EXCELLENT: Features are highly diverse (low similarity)")
elif similarities.mean() < 0.6:
    print("✓ GOOD: Features have good diversity")
elif similarities.mean() < 0.8:
    print("⚠ OK: Features have moderate diversity")
else:
    print("⚠ WARNING: Features are very similar (may indicate issues)")

if dead_features == 0:
    print("✓ GOOD: No dead feature dimensions")
elif dead_features < 100:
    print(f"⚠ OK: {dead_features} dead dimensions (acceptable)")
else:
    print(f"⚠ WARNING: {dead_features} dead dimensions")

print(f"\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"Graphs inspected:     {len(graph_stats)}")
print(f"Total nodes:          {all_features.shape[0]}")
print(f"Feature dimension:    {all_features.shape[1]} (ResNet50)")
print(f"Avg nodes per graph:  {np.mean(node_counts):.1f}")
print(f"Avg edges per graph:  {np.mean(edge_counts):.1f}")
print(f"Feature mean:         {all_features.mean():.6f}")
print(f"Feature std:          {all_features.std():.6f}")
print(f"Feature diversity:    {similarities.mean():.4f} (cosine sim)")
print("="*70)

print("\n✓ Feature inspection complete!")
print("Ready to train transformer when all graphs are built.")

GRAPH FEATURE INSPECTION - SUBSET 1

Total graphs available: 850
Inspecting 20 random graphs...

PER-GRAPH STATISTICS
Graph ID                                       Nodes    Edges  Feat Mean   Feat Std
----------------------------------------------------------------------
7de918b1fabd1ad9594666ebd8fd0652                 104      320     0.3596     0.4194
530fe0124e17f2f3a98c53ffe86bf421                 250      751     0.3627     0.4416
60f316434ad13b545269024f949698cc                  91      268     0.3342     0.4117
a10ba9a0d61fd41d9ca9aae25788504c                 165      488     0.3544     0.4169
6d5c07d90c18669e961483005a6a8a7b                 131      388     0.3622     0.4077
e3e660456c62fb96e17658af48b3e7e4                 153      455     0.3498     0.4060
55f1501ba2898b8842543c1aeb28c891                 140      415     0.3546     0.4233
a5e22243152d5cc871983364ae7b6e95                 189      538     0.3857     0.4360
42cb627feeceb770537e6c322c763dbf                  90   

AttributeError: 'numpy.ndarray' object has no attribute 'median'

In [16]:
#!/usr/bin/env python3
"""
Evaluate GTP Baseline Model and Calculate QWK
"""
import torch
import numpy as np
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report, accuracy_score
from torch.utils.data import DataLoader
from utils.dataset import GraphDataset
from helper import collate
from models.GraphTransformer import Classifier
from collections import OrderedDict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# CONFIGURATION
# ============================================================
VAL_FILE = '/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/scripts/val_set.txt'
TRAIN_FILE = '/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/scripts/train_set.txt'
DATA_PATH = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/foundation_models/feature_extractor/graphs_all/panda"
CHECKPOINT = '/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/graph_transformer/saved_models/GraphCAM_PANDA_WEIGHTED_v2.pth'

N_CLASS = 3
N_FEATURES = 2048  # Changed from 512 to 2048 for ResNet50

# ============================================================
# LOAD VALIDATION DATA
# ============================================================
print("="*70)
print("GTP BASELINE EVALUATION - WEIGHTED MODEL")
print("="*70)
print()

print("Loading validation dataset...")
with open(VAL_FILE, 'r') as f:
    val_ids = [line.strip() for line in f if line.strip()]

dataset_val = GraphDataset(DATA_PATH, val_ids, site='panda')
dataloader_val = DataLoader(
    dataset_val, 
    batch_size=1, 
    collate_fn=collate, 
    num_workers=0, 
    shuffle=False
)

print(f"✓ Validation samples: {len(dataset_val)}")
print()

# ============================================================
# CALCULATE CLASS WEIGHTS
# ============================================================
print("Calculating class weights...")
with open(TRAIN_FILE, 'r') as f:
    train_labels = [int(line.split()[1]) for line in f if line.strip() and len(line.split()) == 2]

class_counts = np.bincount(train_labels)
class_weights = 1.0 / (class_counts + 1e-6)
class_weights = class_weights / class_weights.sum() * 3
class_weights = torch.FloatTensor(class_weights).cuda()

print(f"✓ Class weights: {class_weights.cpu().numpy()}")
print()

# ============================================================
# LOAD MODEL
# ============================================================
print(f"Loading checkpoint: {CHECKPOINT}")

# Load model WITH class weights
model = Classifier(
    n_class=N_CLASS, 
    n_features=N_FEATURES, 
    class_weights=class_weights
)

# Load state dict
state_dict = torch.load(CHECKPOINT, map_location=device)

# Remove 'module.' prefix if present (from DataParallel)
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace('module.', '')
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model = model.to(device).eval()

print("✓ Model loaded and set to eval mode")
print()

# ============================================================
# HOOK TO CAPTURE LOGITS
# ============================================================
logits_captured = None

def hook_fn(module, input, output):
    global logits_captured
    if hasattr(output, 'shape') and len(output.shape) == 2 and output.shape[1] == 3:
        logits_captured = output.clone()

# Register hook on final linear layer
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        module.register_forward_hook(hook_fn)

# ============================================================
# RUN EVALUATION
# ============================================================
print("="*70)
print("RUNNING EVALUATION")
print("="*70)
print()

all_preds = []
all_labels = []
skipped = 0

with torch.no_grad():
    for i, sample in enumerate(dataloader_val):
        if sample is None:
            skipped += 1
            continue
        
        try:
            # Extract data
            img = sample["image"][0] if isinstance(sample["image"], list) else sample["image"]
            adj = sample["adj_s"][0] if isinstance(sample["adj_s"], list) else sample["adj_s"]
            label = sample["label"][0] if isinstance(sample["label"], list) else sample["label"]
            
            # Prepare tensors
            img = img.unsqueeze(0).float().to(device)
            adj = adj.unsqueeze(0).float().to(device)
            mask = torch.ones(1, img.size(1)).to(device)
            label_tensor = torch.tensor([label], dtype=torch.long).to(device)
            
            # Forward pass
            logits_captured = None
            model(img, label_tensor, adj, mask)
            
            # Get prediction
            if logits_captured is not None:
                pred = logits_captured.argmax(1).item()
                all_preds.append(pred)
                all_labels.append(label)
            
            # Progress update
            if (i + 1) % 200 == 0:
                print(f"  Processed {i+1}/{len(dataloader_val)} samples")
        
        except Exception as e:
            skipped += 1
            continue

print()
print(f"✓ Evaluation complete!")
print(f"  Predictions: {len(all_preds)}")
print(f"  Skipped: {skipped}")
print()

# ============================================================
# CALCULATE METRICS
# ============================================================
print("="*70)
print("BASELINE RESULTS")
print("="*70)
print()

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Calculate metrics
acc = accuracy_score(all_labels, all_preds)
qwk = cohen_kappa_score(all_labels, all_preds, weights='quadratic')

print(f"Validation Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print(f"\n★★★ BASELINE QWK: {qwk:.4f} ★★★")
print()

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])
print("Confusion Matrix:")
print("         Pred0  Pred1  Pred2")
print(f"Actual 0: {cm[0][0]:4d}   {cm[0][1]:4d}   {cm[0][2]:4d}")
print(f"Actual 1: {cm[1][0]:4d}   {cm[1][1]:4d}   {cm[1][2]:4d}")
print(f"Actual 2: {cm[2][0]:4d}   {cm[2][1]:4d}   {cm[2][2]:4d}")
print()

# Classification report
print("Classification Report:")
print(classification_report(
    all_labels, 
    all_preds, 
    target_names=['Class 0 (Background)', 'Class 1 (Benign)', 'Class 2 (Cancerous)'], 
    digits=4,
    zero_division=0
))

# ============================================================
# GRADE ASSESSMENT
# ============================================================
print("="*70)
print("GRADE ASSESSMENT")
print("="*70)
print()

target_b_plus = qwk * 1.05
target_a = qwk * 1.10

print(f"★ BASELINE QWK: {qwk:.4f}")
print(f"  B+ target (5% improvement):  {target_b_plus:.4f}")
print(f"  A target (10% improvement):  {target_a:.4f}")
print()

if qwk >= 0.46:
    grade = "A"
    print("🎉🎉🎉 EXCELLENT! A-GRADE BASELINE! 🎉🎉🎉")
    print("Your baseline alone qualifies for A grade!")
elif qwk >= 0.42:
    grade = "B+"
    print("✓✓ GOOD! B+ GRADE BASELINE!")
    print("Try one improvement method for A grade")
elif qwk >= 0.35:
    grade = "B"
    print("✓ DECENT! B GRADE BASELINE")
    print("Need improvement for higher grade")
else:
    grade = "< B"
    print("⚠ Below expected - may need debugging")

print("="*70)
print()

# ============================================================
# SAVE RESULTS
# ============================================================
results_file = 'baseline_qwk_results.txt'
print(f"Saving results to: {results_file}")

with open(results_file, 'w') as f:
    f.write("="*70 + "\n")
    f.write("GTP BASELINE EVALUATION RESULTS\n")
    f.write("="*70 + "\n\n")
    
    f.write("Method: Graph Transformer (GTP)\n")
    f.write("Backbone: ResNet50 (ImageNet pretrained, frozen)\n")
    f.write("Feature Dimension: 2048\n")
    f.write(f"Validation Samples: {len(all_labels)}\n\n")
    
    f.write(f"★ BASELINE QWK: {qwk:.4f}\n")
    f.write(f"  Validation Accuracy: {acc:.4f} ({acc*100:.2f}%)\n")
    f.write(f"  Grade: {grade}\n\n")
    
    f.write("Confusion Matrix:\n")
    f.write("         Pred0  Pred1  Pred2\n")
    f.write(f"Actual 0: {cm[0][0]:4d}   {cm[0][1]:4d}   {cm[0][2]:4d}\n")
    f.write(f"Actual 1: {cm[1][0]:4d}   {cm[1][1]:4d}   {cm[1][2]:4d}\n")
    f.write(f"Actual 2: {cm[2][0]:4d}   {cm[2][1]:4d}   {cm[2][2]:4d}\n\n")
    
    f.write("Grade Targets:\n")
    f.write(f"  B+ (5% improvement):  {target_b_plus:.4f}\n")
    f.write(f"  A (10% improvement):  {target_a:.4f}\n")

print(f"✓ Results saved to: {results_file}")
print()
print("="*70)
print("EVALUATION COMPLETE!")
print("="*70)

ModuleNotFoundError: No module named 'utils'